# Grupo

### Matheus Lobo Vaz Araújo
### Paulo Fernando Barros Cavalcanti

# Resumo do Funcionamento do Código

### Contextualização
Neste projeto, escolhemos utilizar um dos TCCs de integrantes do grupo. Acreditamos que seria uma ideia criativa e curiosa explorar como um modelo de linguagem poderia resumir um documento escrito ao longo de um ano de pesquisa e refinamento. Abaixo, detalhamos as etapas do código desenvolvido:

---

### Etapas do Código

1. **Leitura do Documento PDF**:
   O código utiliza a biblioteca `PyPDF2` para abrir e extrair o texto do arquivo PDF do TCC. Cada página é lida e concatenada em uma única string que representa o conteúdo completo do documento.

2. **Divisão do Texto (Splitting)**:
   Para facilitar a indexação e o processamento do texto, o documento é dividido em fragmentos menores (chunks) utilizando o `CharacterTextSplitter` da biblioteca LangChain. Esses fragmentos têm 300 caracteres, com uma sobreposição de 50 caracteres para manter a continuidade do contexto.

3. **Criação de um Vector Store**:
   Os fragmentos de texto são convertidos em vetores utilizando embeddings da OpenAI (com a classe `OpenAIEmbeddings`). Esses vetores são então armazenados em um repositório vetorial (Vector Store) utilizando a implementação `FAISS`. Esse repositório permite recuperar informações relevantes do texto com base em consultas.

4. **Recuperação de Informações (Retrieval)**:
   A função `retrieve` utiliza o Vector Store para buscar fragmentos de texto que sejam relevantes para uma consulta específica (query). Esse processo é realizado com um retriever configurado para acessar o conteúdo do repositório vetorial.

5. **Geração de Resumos e Respostas**:
   Para gerar respostas, utilizamos um modelo de linguagem da OpenAI configurado com `ChatOpenAI` (por exemplo, `gpt-3.5-turbo`). Combinamos o modelo com o retriever utilizando a classe `RetrievalQA`. Assim, o sistema primeiro localiza os fragmentos mais relevantes e, em seguida, o modelo gera um resumo ou resposta baseado nesses fragmentos.

6. **Execução e Testes**:
   O código é configurado para realizar uma consulta ao conteúdo do TCC. No exemplo, a query é: *"Resuma o conteúdo do TCC Final - Paulo Fernando de B. Cavalcanti."*. O sistema recupera os fragmentos relevantes e, em seguida, utiliza o modelo para gerar um resumo detalhado.

---

### Observações Importantes

- **Chave de API**: A chave API vai estar apagada no codigo final encontrado no repositorio.

---


In [1]:
# Importando bibliotecas necessárias
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import PyPDF2

# Etapa 1: Leitura do documento PDF
def read_pdf(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = ""
        for page in reader.pages:
            text += page.extract_text()
    return text


In [2]:
# Etapa 2: Splitting do Documento
document_path = "TCC Final - Paulo Fernando de B. Cavalcanti.pdf"
document_text = read_pdf(document_path)
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_texts = text_splitter.split_text(document_text)

In [3]:
# Etapa 3: Criação do Vector Store
api_key = " "  # Substitua por sua chave de API da OpenAI
embeddings = OpenAIEmbeddings(client=None, openai_api_key=api_key)
vector_store = FAISS.from_texts(split_texts, embeddings)

C:\Users\mlva\AppData\Local\Temp\ipykernel_16564\1783873490.py:3: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(client=None, openai_api_key=api_key)


In [4]:
# Etapa 4: Retrieval
def retrieve(query):
    retriever = vector_store.as_retriever()
    return retriever.get_relevant_documents(query)

In [5]:
# Etapa 5: Geração de Respostas
llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=vector_store.as_retriever())

def generate_summary(query):
    """
    Gera um resumo baseado na query aplicada ao conteúdo do TCC.
    """
    return qa_chain.run(query)

C:\Users\mlva\AppData\Local\Temp\ipykernel_16564\128849060.py:2: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=api_key)


In [7]:
query = "Resuma o conteúdo do TCC Final - Paulo Fernando de B. Cavalcanti."
retrieved_docs = retrieve(query)
print("Documentos recuperados:")
for doc in retrieved_docs:
    print(doc.page_content)

Documentos recuperados:
CESAR SCHOOL  
GRADUAÇÃO EM CIÊNCIA DA COMPUTAÇÃO  
  
 
PAULO FERNANDO DE BARROS CAVALCANTI  
 
 
 
ANÁLISE DE PONTOS CRÍTICOS DA MIGRAÇÃO DE 
UM SISTEMA MONOLÍTICO PARA UMA ARQUITETURA 
DE MICROSSERVIÇOS UTILIZANDO A TÉCNICA DO 
ESTRANGULAMENTO  
 
 
 
 
 
 
 
RECIFE  
2023   
 
PAULO  FERNANDO  DE BARROS  CAVALCANTI  
 
 
 
 
 
 
ANÁLISE DE PONTOS CRÍTICOS DA MIGRAÇÃO DE 
UM SISTEMA MONOLÍTICO PARA UMA ARQUITETURA 
DE MICROSSERVIÇOS UTILIZANDO A TÉCNICA DO 
ESTRANGULAMENTO  
 
 
 
 
 
Trabalho de Conclusão de Curso 
apresentado ao Curso de Graduação do 
Centro de Estudos e Sistemas Educacionais 
do Recife – CESAR School, como requisito 
para a obtenção do título de Bacharel em 
Ciência da Computação.  
Orientação: Prof.  Carlos Diego Cavalcanti 
Pereira  
 
 
 
 
 
RECIFE  
2023   
 
Análise de Pontos Críticos da Migração de um Sistema 
Monolítico para uma Arquitetura de Microsserviços 
Utilizando a Técnica do Estrangulamento  
 
 
Resumo : A contribuição des

In [8]:
print("\nResumo gerado:")
summary = generate_summary(query)
print(summary)


Resumo gerado:
O Trabalho de Conclusão de Curso de Paulo Fernando de Barros Cavalcanti abordou a análise dos pontos críticos e desafios enfrentados durante a migração de um sistema monolítico para uma arquitetura de microsserviços, utilizando a técnica do estrangulamento. O estudo revisou a literatura sobre o tema e destacou etapas essenciais nesse processo de migração. O trabalho destaca a importância do conhecimento prévio dos obstáculos e áreas sensíveis durante a execução dessa transição arquitetural. O objetivo foi contribuir com conhecimentos relevantes para pesquisadores iniciantes e profissionais da área que buscam entender e superar as dificuldades associadas à migração de sistemas.
